In [57]:
%matplotlib inline

import numpy as np
import pandas as pd

In [58]:
df = pd.read_table('lpjguess/output/sp_lai.out', delim_whitespace=True)
df.head()

,Lon,Lat,Year,Stand,Patch,TeBE_tm,TeBE_itm,TeBE_itscl,TeBS_tm,TeBS_itm,TeE_s,TeR_s,TeNE,BBS_itm,BBE_itm,BE_s,C3G,Total
0,-70.75,-26.25,1950,0,0,0.0,0.0,0.0081,0.0,0.0,0.0492,0.0000,0.0,0.0,0.0,0.0,0.0000,0.0574
1,-70.75,-26.25,1950,0,1,0.0,0.0,0.0000,0.0,0.0,0.0072,0.0962,0.0,0.0,0.0,0.0,0.0001,0.1034
2,-70.75,-26.25,1950,0,2,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0000,0.0000
3,-70.75,-26.25,1950,0,3,0.0,0.0,0.0000,0.0,0.0,0.0633,0.0324,0.0,0.0,0.0,0.0,0.0011,0.0969
4,-70.75,-26.25,1950,0,4,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0000,0.0000


In [59]:
def calc_fpc(lai):
    """Calculate FPC using the LPJ-GUESS method
    """
    return (1.0 - np.exp(-0.5 * lai)) * 100


In [60]:
# filter non-landform data (lf 0) and remove patch var
df = df[df.Stand > 0]
del df['Patch']
df = df.rename(columns={'Stand': 'lf_id'})

In [61]:
df_grp = df.groupby(['Lon', 'Lat', 'Year', 'lf_id']).mean()
df_grp = df_grp.apply(calc_fpc, 1)
df_grp.head()

TeBE_tm  TeBE_itm  TeBE_itscl  TeBS_tm  TeBS_itm  \
Lon    Lat    Year lf_id                                                     
-70.75 -26.25 1950 110        0.0       0.0    0.030995      0.0       0.0   
                   131        0.0       0.0    0.219758      0.0       0.0   
                   132        0.0       0.0    0.167859      0.0       0.0   
                   133        0.0       0.0    0.085963      0.0       0.0   
                   134        0.0       0.0    0.005000      0.0       0.0   

                             TeE_s     TeR_s  TeNE  BBS_itm  BBE_itm  BE_s  \
Lon    Lat    Year lf_id                                                     
-70.75 -26.25 1950 110    0.938568  1.537065   0.0      0.0      0.0   0.0   
                   131    0.142898  2.454378   0.0      0.0      0.0   0.0   
                   132    0.555452  5.022484   0.0      0.0      0.0   0.0   
                   133    1.200733  3.140625   0.0      0.0      0.0   0.0   
                   134    0.136906  4.257702   0.0      0.0      0.0   0.0   

                               C3G     Total  
Lon    Lat    Year lf_id                      
-70.75 -26.25 1950 110    0.301544  2.785470  
                   131    0.222752  3.022384  
                   132    0.666767  6.335412  
                   133    4.739029  8.915269  
                   134    0.609137  4.975934

In [160]:
df_agg = df_grp.loc[:,['C3G', 'Total']]
df_agg['shrub'] = df_grp[['BE_s','TeR_s','TeE_s']].sum(axis=1)
df_agg['tree']  = df_grp[['TeBE_tm','TeBE_itm','TeBE_itscl','TeBS_itm','TeNE','BBS_itm','BBE_itm']].sum(axis=1)

df_agg = df_agg[['C3G','shrub','tree','Total']]
df_agg.columns = ['grass', 'shrub', 'tree', 'total']

for v in ['grass', 'shrub', 'tree', 'total']:
    x = df_agg[v].reset_index().set_index(['Year', 'lf_id'])
    del x['Lon'], x['Lat']
    x = x.mean(level=1).round(2).T
    x.to_csv(f'fpc_{v}.csv', index=False)

In [162]:
!ls

Pipfile           environment.sh    fpc_tree.csv      scripts
Pipfile.lock      forcings          landlab           tmp
README.md         fpc_grass.csv     looper.sh
Untitled.ipynb    fpc_shrub.csv     lpjguess
dummy.csv         fpc_total.csv     lpjguesstools.log


In [163]:
!head fpc_grass.csv

110,131,132,133,134,140,160
2.02,1.52,1.32,1.23,1.89,1.03,1.38


In [166]:
index_cols = ['Lat', 'Lon', 'Year', 'Stand', 'Patch'] 
tree_cols = ['TeBE_tm','TeBE_itm','TeBE_itscl','TeBS_itm','TeNE','BBS_itm','BBE_itm']
shrub_cols = ['BE_s','TeR_s','TeE_s']
grass_cols = ['C3G']
total_col = ['Total']

ftype='lai'
pft_class = 'tree'


requested_cols = tree_cols

In [174]:
df = pd.read_table('lpjguess/output/sp_lai.out', delim_whitespace=True)[index_cols + requested_cols]
df = df[df.Stand > 0]
del df['Patch']
print(df.head())
df_grp = df.groupby(['Lon', 'Lat', 'Year', 'Stand']).mean()
df_grp = df_grp.apply(calc_fpc, 1).sum(axis=1)

print(df_grp.head())

df = df_grp.reset_index().set_index(['Year', 'Stand'])

print(df)

del df['Lon'], df['Lat']
df = df.mean(level=1).round(2).T
print(df)
#x.to_csv(f'fpc_{v}.csv', index=False)

     Lat    Lon  Year  Stand  TeBE_tm  TeBE_itm  TeBE_itscl  TeBS_itm  TeNE  \
5 -26.25 -70.75  1950    110      0.0       0.0      0.0000       0.0   0.0   
6 -26.25 -70.75  1950    110      0.0       0.0      0.0000       0.0   0.0   
7 -26.25 -70.75  1950    110      0.0       0.0      0.0000       0.0   0.0   
8 -26.25 -70.75  1950    110      0.0       0.0      0.0031       0.0   0.0   
9 -26.25 -70.75  1950    110      0.0       0.0      0.0000       0.0   0.0   

   BBS_itm  BBE_itm  
5      0.0      0.0  
6      0.0      0.0  
7      0.0      0.0  
8      0.0      0.0  
9      0.0      0.0  
Lon     Lat     Year  Stand
-70.75  -26.25  1950  110      0.030995
                      131      0.219758
                      132      0.167859
                      133      0.085963
                      134      0.005000
dtype: float64
              Lon    Lat         0
Year Stand                        
1950 110   -70.75 -26.25  0.030995
     131   -70.75 -26.25  0.219758
     132  